In [ ]:
import os
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, default_data_collator

In [ ]:
hf_token = "..."

# load datasets
eng_data = pd.read_excel("eng_data.xlsx")

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"
pipe = pipeline("text-generation", model=model, trust_remote_code=True, token=hf_token)

# generate responses with pipeline
eng_responses = []
for _, row in eng_data.iterrows():
    text = row["text"]
    prompt = f"<INST>Given the following text, please generate a prompt that it could be a response for:\n\"{text}\"</INST>"

    result = pipe(prompt, max_new_tokens=50)
    generated = result.split("</INST>")[-1].strip()

    eng_responses.append(generated)
eng_data["prompt"] = eng_responses
eng_train, eng_test = train_test_split(eng_data, test_size=0.2, random_state=42, stratify=eng_data["normalized_level"])
eng_data.to_excel("eng_data_with_prompt.xlsx", index=False)

In [ ]:
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
tokenizer_name_or_path = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
# Load model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="cuda", token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token